# Trabajo de Fin de Grado: Aplicación de Inteligencia artificial a la predicción de los efectos de la radiación en sistemas digitales complejos

### Pablo Darós Pallarés

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [4]:
df = pd.read_csv('data/radecs_complete_data.csv', delimiter=";")

print(df.shape)

(4159, 22)


En este codigo se busca crear columnas nuevas con variables categóricas para cada SEE. Este acercamiento resultó ser innecesario, ya que un componente puede sufrir varios SEE a la vez. 

In [ ]:
df = pd.read_csv('data/radecs_complete_data.csv', delimiter=";")

# Encontrar los valores posibles de cada SEE
see_columns = ['SEU', 'SET', 'SEFI', 'SEL', 'SEB', 'SEGR']
for column in see_columns:
    unique_combinations = df[[column]].apply(lambda row: '_'.join(row.dropna().values), axis=1).unique()
    unique_combinations = [combo for combo in unique_combinations if combo]

    print(column, "nº of elements: ", len(unique_combinations), '\n combinations: ', unique_combinations)

# Cada valor del SEE tiene un numero asignado
seu_mapping = {
    'Alpha': 0,
    'e': 1,
    'H': 2,
    'H,P': 3,
    'H,L': 4,
    'H,N,P': 5,
    'H,N': 6,
    'L': 7,
    'L,H': 8,
    'N': 9,
    'P': 10,
    'P,Alpha': 11,
    'P,N': 12,
    'P,N,Alpha': 13
}

set_mapping = {
    'H': 14,
    'P': 15,
    'H,N': 16,
    'N': 17,
    'H,P': 18,
    'H,L': 19,
    'L': 20,
    'L,P': 21
}

sefi_mapping = {
    'H': 22,
    'P': 23,
    'H,N': 24,
    'N': 25,
    'P,N': 26,
    'H,P': 27,
    'H,L': 28,
    'H,N,P': 29,
    'H,P,L': 30
}

sel_mapping = {
    'H': 31,
    'P': 32,
    'H,N': 33,
    'H,P': 34,
    'N': 35,
    'P,N': 36,
    'H,L': 37,
    'H,P,N': 38,
    'L': 39
}

seb_mapping = {
    'H': 40,
    'P': 41,
    'N': 42,
    'H,P': 43
}

segr_mapping = {
    'H': 44,
    'P': 45,
    'N': 46
}

# Asignar los valores a columnas nuevas
df['SEU_Value'] = df['SEU'].map(seu_mapping)
df['SET_Value'] = df['SET'].map(set_mapping)
df['SEFI_Value'] = df['SEFI'].map(sefi_mapping)
df['SEL_Value'] = df['SEL'].map(sel_mapping)
df['SEB_Value'] = df['SEB'].map(seb_mapping)
df['SEGR_Value'] = df['SEGR'].map(segr_mapping)

# Encontrar los valores posibles de cada SEE
see_columns = ['SEU_Value', 'SET_Value', 'SEFI_Value', 'SEL_Value', 'SEB_Value', 'SEGR_Value']
for column in see_columns:
    unique_combinations = df[[column]].apply(lambda row: '_'.join(map(str, row.dropna().values)), axis=1).unique()
    unique_combinations = [combo for combo in unique_combinations if combo]

    print(column, "nº of elements: ", len(unique_combinations), '\n combinations: ', unique_combinations)

print(df)

En este codigo se agrupan todos los SEE bajo una misma columna

In [ ]:
df = pd.read_csv('data/radecs_complete_data.csv', delimiter=";")

# Cada valor del SEE tiene un numero asignado
seu_mapping = {
    'Alpha': 0,
    'e': 1,
    'H': 2,
    'H,P': 3,
    'H,L': 4,
    'H,N,P': 5,
    'H,N': 6,
    'L': 7,
    'L,H': 8,
    'N': 9,
    'P': 10,
    'P,Alpha': 11,
    'P,N': 12,
    'P,N,Alpha': 13
}

set_mapping = {
    'H': 14,
    'P': 15,
    'H,N': 16,
    'N': 17,
    'H,P': 18,
    'H,L': 19,
    'L': 20,
    'L,P': 21
}

sefi_mapping = {
    'H': 22,
    'P': 23,
    'H,N': 24,
    'N': 25,
    'P,N': 26,
    'H,P': 27,
    'H,L': 28,
    'H,N,P': 29,
    'H,P,L': 30
}

sel_mapping = {
    'H': 31,
    'P': 32,
    'H,N': 33,
    'H,P': 34,
    'N': 35,
    'P,N': 36,
    'H,L': 37,
    'H,P,N': 38,
    'L': 39
}

seb_mapping = {
    'H': 40,
    'P': 41,
    'N': 42,
    'H,P': 43
}

segr_mapping = {
    'H': 44,
    'P': 45,
    'N': 46
}

# Añadir los posibles valores a una lista
def map_see(row):
    see_values = []
    if row['SEU'] in seu_mapping:
        see_values.append(f"SEU:{row['SEU']}")
    if row['SET'] in set_mapping:
        see_values.append(f"SET:{row['SET']}")
    if row['SEFI'] in sefi_mapping:
        see_values.append(f"SEFI:{row['SEFI']}")
    if row['SEL'] in sel_mapping:
        see_values.append(f"SEL:{row['SEL']}")
    if row['SEB'] in seb_mapping:
        see_values.append(f"SEB:{row['SEB']}")
    if row['SEGR'] in segr_mapping:
        see_values.append(f"SEGR:{row['SEGR']}")

    return see_values

# Crear una columna nueva con todos los posibles valores
df['SEE'] = df.apply(map_see, axis=1)

unique_df = df

print(df.columns)


En este codigo se reealiza one hot encoding para transformar las variables en categoricas

In [ ]:
df = pd.read_csv('data/radecs_complete_data.csv', delimiter=";")
# Realizar one hot encoding
df_one_hot = pd.get_dummies(df, columns=['SEU', 'SET', 'SEFI', 'SEL', 'SEB', 'SEGR'])

print(df_one_hot)

Este codigo es para la generación de clusters

In [ ]:
df = pd.read_csv('data/radecs_complete_data.csv', delimiter=";")

# Se crea el vectorizador TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['Type'].fillna(''))  # Asegúrate de que no hay NaNs
kmeans = KMeans(n_clusters=10, random_state=0)
kmeans.fit(X)

words = vectorizer.get_feature_names_out()

centroids = kmeans.cluster_centers_

# Se muestran las 10 palabras mas relevantes
for i, centroid in enumerate(centroids):
    print(f"\nCluster {i}:")
    sorted_words = sorted(zip(centroid, words), reverse=True, key=lambda x: x[0])

    word_count = sum(1 for value, word in sorted_words if value > 0.01)
    print(f"Palabras totales con importancia > 0.01: {word_count}")
    
    top_words = sorted_words[:3]
    for score, word in top_words:
        print(f"{word} ({score:.2f})")

df_cluster = df

In [ ]:
print(unique_df.columns)

print(df_one_hot.columns)

Se eliminan las columnas de los SEE en el df único

In [ ]:
SEES_list = ['SEU','SET','SEFI','SEL','SEB','SEGR']


for see in SEES_list:
    unique_df.drop(see, axis=1, inplace=True)


Se generan los archivos excel para todos los df creados

In [ ]:
unique_df.to_csv('radecs_unique_label.csv', index=False, sep=',', encoding='utf-8')

df_one_hot.to_csv('radecs_one_hot_label.csv', index=False, sep=',', encoding='utf-8')

df_cluster.to_csv('radecs_cluster.csv', index=False, sep=',', encoding='utf-8')